In [1]:
import os

In [2]:
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
import config
import math
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import random
import cityscapes_labels
import utils
from generator import *

from models import *
from keras import losses, metrics, optimizers
from metrics import precision, dice_coef

import time
import pylab as pl
from IPython import display

Using TensorFlow backend.


In [4]:
dataset_path = config.data_path()
datagen = CityscapesFlowGenerator(dataset_path, prev_skip=0, flow_with_diff=True)

/home/mlyko/data/cityscapes/
Cityscapes: shuffling dataset
Cityscapes: train 2975 files
Cityscapes: shuffling dataset
Cityscapes: val 500 files
Cityscapes: shuffling dataset
Cityscapes: test 1525 files
training samples 2975, validating samples 500, test samples 1525


In [5]:
target_size = 256, 512
batch_size = 2
epochs = 10
eval_batch_size = 5

In [8]:
models = [
 #   (SegNet(target_size, datagen.n_classes),  '/home/mlyko/weights/city/SegNet/spat_drop04.h5'),
  #  (SegNetWarpDiff123(target_size, datagen.n_classes), '/home/mlyko/weights/city/SegNetWarpDiff/diff_p0_w123_s04_2.h5'),
   # (SegNetWarpDiff123(target_size, datagen.n_classes),'../../weights/city/rel/SegNetWarpDiff123/diff_p0_w123_s04_norm_eq_hist_dec_0001_g2.h5'),
    (MobileUNetWarp(target_size, datagen.n_classes), '/home/mlyko/weights/city/rel/MobileUNetWarp/mob_unet_b6_warp1_spat0.2.h5'),
]

for m, w in models:
    print("--loading model %s" % m.name)
    m.compile()
    m.k.load_weights(w)
    print(m.k.metrics_names)
    print("--evaluating model")
    
    prediction = m.k.evaluate_generator(
        generator=datagen.flow('val', eval_batch_size, target_size),
        steps=datagen.steps_per_epoch('val', eval_batch_size),
    )

    result = dict(zip(m.k.metrics_names, prediction))
    print(result)
    

--loading model MobileUNetWarp
['loss', 'precision', 'recall', 'f1_score', 'categorical_accuracy', 'mean_iou']
--evaluating model
[1.5700637578964234, 0.9118392097949982, 0.48399852961301804, 0.6315213459730148, 0.7572214961051941, 0.7617437011003494]


{'f1_score': 0.6315213459730148, 'loss': 1.5700637578964234, 'recall': 0.48399852961301804, 'precision': 0.9118392097949982, 'mean_iou': 0.7617437011003494, 'categorical_accuracy': 0.7572214961051941}


In [13]:
m,w= (SegNetWarpDiff123(target_size, datagen.n_classes),'../../weights/city/rel/SegNetWarpDiff123/diff_p0_w123_s04_norm_eq_hist_dec_0001_g2.h5')
print("--loading model %s" % m.name)
m.compile()
m.k.load_weights(w)
print(m.k.metrics_names)
print("--evaluating model")

prediction = m.k.evaluate_generator(
    generator=datagen.flow('val', eval_batch_size, target_size),
    steps=datagen.steps_per_epoch('val', eval_batch_size),
)

result = dict(zip(m.k.metrics_names, prediction))
print(result)

--loading model SegNetWarpDiff123
['loss', 'precision', 'recall', 'f1_score', 'categorical_accuracy', 'mean_iou']
--evaluating model
{'f1_score': 0.6166577243804932, 'loss': 1.6053984427452088, 'recall': 0.4667716193199158, 'precision': 0.9118875598907471, 'mean_iou': 0.7582826566696167, 'categorical_accuracy': 0.7816843265295028}


In [6]:
m,w= (MobileUNetWarp(target_size, datagen.n_classes),'../../weights/city/rel/MobileUNetWarp/w_1_b5_r_up5bn.h5')
print("--loading model %s" % m.name)
m.compile()
m.k.load_weights(w)
print(m.k.metrics_names)
print("--evaluating model")

prediction = m.k.evaluate_generator(
    generator=datagen.flow('val', eval_batch_size, target_size),
    steps=datagen.steps_per_epoch('val', eval_batch_size),
)

result = dict(zip(m.k.metrics_names, prediction))
print(result)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data_new (InputLayer)           (None, 256, 512, 3)  0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 128, 256, 32) 864         data_new[0][0]                   
__________________________________________________________________________________________________
conv_0_bn (BatchNormalization)  (None, 128, 256, 32) 128         conv_0[0][0]                     
__________________________________________________________________________________________________
conv_0_relu (Activation)        (None, 128, 256, 32) 0           conv_0_bn[0][0]                  
__________________________________________________________________________________________________
conv_dw_1 

['loss', 'precision', 'recall', 'f1_score', 'categorical_accuracy', 'mean_iou']
--evaluating model
{'f1_score': 0.6291570138931274, 'loss': 1.6211587715148925, 'recall': 0.4836016085743904, 'precision': 0.902803116440773, 'mean_iou': 0.7557701921463013, 'categorical_accuracy': 0.7447564989328385}


In [6]:

datagen_without_flow = CityscapesGenerator(dataset_path)
m,w= (MobileUNet(target_size, datagen_without_flow.n_classes),'/home/mlyko/weights/city/rel/MobileUNet/mob_unet_b6_spatdrop.h5')
print("--loading model %s" % m.name)
m.compile()
m.k.load_weights(w)
print(m.k.metrics_names)
print("--evaluating model")

prediction = m.k.evaluate_generator(
    generator=datagen_without_flow.flow('val', eval_batch_size, target_size),
    steps=datagen_without_flow.steps_per_epoch('val', eval_batch_size),
)

result = dict(zip(m.k.metrics_names, prediction))
print(result)

/home/mlyko/data/cityscapes/
Cityscapes: shuffling dataset
Cityscapes: train 2975 files
Cityscapes: shuffling dataset
Cityscapes: val 500 files
Cityscapes: shuffling dataset
Cityscapes: test 1525 files
training samples 2975, validating samples 500, test samples 1525
--loading model MobileUNet
['loss', 'precision', 'recall', 'f1_score', 'categorical_accuracy', 'mean_iou']
--evaluating model
{'f1_score': 0.6339343905448913, 'loss': 1.5997431993484497, 'recall': 0.489443176984787, 'precision': 0.9027679562568665, 'mean_iou': 0.7672078520059585, 'categorical_accuracy': 0.7744514751434326}


In [7]:
datagen_without_flow = CityscapesGenerator(dataset_path)
m,w= (SegNet(target_size, datagen_without_flow.n_classes),'/home/mlyko/weights/city/SegNet/spat_drop04.h5')
print("--loading model %s" % m.name)
m.compile()
m.k.load_weights(w)
print(m.k.metrics_names)
print("--evaluating model")

prediction = m.k.evaluate_generator(
    generator=datagen_without_flow.flow('val', eval_batch_size, target_size),
    steps=datagen_without_flow.steps_per_epoch('val', eval_batch_size),
)

result = dict(zip(m.k.metrics_names, prediction))
print(result)

/home/mlyko/data/cityscapes/
Cityscapes: shuffling dataset
Cityscapes: train 2975 files
Cityscapes: shuffling dataset
Cityscapes: val 500 files
Cityscapes: shuffling dataset
Cityscapes: test 1525 files
training samples 2975, validating samples 500, test samples 1525
--loading model SegNet
['loss', 'precision', 'recall', 'f1_score', 'categorical_accuracy', 'mean_iou']
--evaluating model
{'f1_score': 0.01825033458415419, 'loss': 14.39424946784973, 'recall': 0.013585748542100191, 'precision': 0.0279078955296427, 'mean_iou': 0.02677584663964808, 'categorical_accuracy': 0.006486343376163859}


In [ ]:
f, arr= plt.subplots(3,2)
f.set_size_inches(14,14)

for imgBatch, labelBatch in datagen.flow('val',1,target_size):
    predictions = []
    colored = []

    imgNew = imgBatch[1][0]
    predicting_image = cv2.cvtColor( imgNew,  cv2.COLOR_BGR2RGB)
    
    i = 0
    print(models[i][0].name)
    predictions.append(models[i][0].k.predict(np.array([imgNew]), batch_size, verbose=1))
    colored.append(datagen.one_hot_to_bgr(predictions[i], target_size, datagen.n_classes, datagen.old_labels))
    
    i += 1
    print(models[i][0].name)
    predictions.append(models[i][0].k.predict(imgBatch, batch_size, verbose=1))
    colored.append(datagen.one_hot_to_bgr(predictions[i], target_size, datagen.n_classes, datagen.old_labels))

    i += 1
    print(models[i][0].name)
    predictions.append(models[i][0].k.predict(imgBatch, batch_size, verbose=1))
    colored.append(datagen.one_hot_to_bgr(predictions[i], target_size, datagen.n_classes, datagen.labels))
    
    i += 1
    print(models[i][0].name)
    predictions.append(models[i][0].k.predict(imgBatch, batch_size, verbose=1))
    colored.append(datagen.one_hot_to_bgr(predictions[i], target_size, datagen.n_classes, datagen.labels))
        

    arr[0][0].imshow(predicting_image)
    arr[0][1].imshow(datagen.one_hot_to_bgr(labelBatch, target_size, datagen.n_classes, datagen.labels))
    
    for p in range(len(models)):
        row = int(math.floor(p / 2)) + 1
        col = p % 2
        arr[row][col].set_title(models[p][0].name)
        arr[row][col].imshow(colored[p])
    
    display.clear_output(wait=True)
    display.display(pl.gcf())
    time.sleep(4.0)

In [ ]:
# test visualize
import keras.backend as K

model = model4
model.summary()
datagen = CityscapesFlowGenerator(dataset_path, prev_skip=0, flow_with_diff=True)

In [ ]:
def layer_to_visualize(img_to_visualize, model, layer, feed=[]):
    inputs = [K.learning_phase()] + model.inputs

    _convout1_f = K.function(inputs, [layer.output])
    def convout1_f(X):
        # The [0] is to disable the training phase flag
        return _convout1_f([0] + X)

    convolutions = convout1_f(img_to_visualize)    
    convolutions = np.squeeze(convolutions)
    convolutions=np.moveaxis(convolutions, 2, 0)
    print ('Shape of conv:', convolutions.shape)
    
    filters = convolutions.shape[0]
    print("convs", filters)
    n = int(np.ceil(np.sqrt(filters)))
        
    # Visualization of each filter of the layer
    fig = plt.figure(figsize=(32,32))
#    ax = fig.add_subplot(1,1)    
    #plt.imshow(convolutions[0], cmap='gray')
    #  flow_bgr=datagen.flow_to_bgr(convolutions, convolutions.shape[1:])
    
    #return
    
    for i in range(filters):
        ax = fig.add_subplot(n,n,i+1)
     #   flow_bgr=datagen.flow_to_bgr(convolutions, convolutions.shape[1:])
        ax.imshow(convolutions[i], cmap='gray')

In [ ]:
for imgBatch, labelBatch in datagen.flow('train',1,target_size):
    imgOld = imgBatch[0]
    imgNew = imgBatch[1][0]    
    
    layers = [model.k.get_layer('data_old'), model.k.get_layer('data_new'), model.k.get_layer('data_flow'), model.k.get_layer('data_diff')]

#    layer_to_visualize(imgBatch, model.k, model.k.get_layer('transformed_flow'), layers)    
#    layer_to_visualize(imgBatch, model.k, model.k.get_layer('flow_down_1'), layers)    
    layer_to_visualize(imgBatch, model.k, model.k.get_layer('activation_35'), layers)    
    
    break

#    layer_to_visualize(np.array([imgOld]), model.k.get_layer('conv2d_27'), model.k)    
 #   layer_to_visualize(np.array([imgOld]), model.k.get_layer('conv2d_19'), model.k)    
    
#    layer_to_visualize(img, model.k.get_layer('warp'), model.k)    

#    layer_to_visualize(to_vis, model.k.get_layer('conv2d_transpose_1'), model.k)    
#    layer_to_visualize(to_vis, model.k.get_layer('conv2d_transpose_4'), model.k)    
    
    
    

In [ ]:
!ls /home/mlyko/data/cityscapes/leftImg8bit/train/

In [ ]:
cityscapes_labels.labels